In [62]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon



terrenos_path = './data/raw_data/terrenos.csv'   
servicios_path = './data/raw_data/servicios.csv' 

terrenos_df = pd.read_csv(terrenos_path)
servicios_df = pd.read_csv(servicios_path)

# Validar columnas necesarias
if 'Nodos' not in terrenos_df.columns or 'Coordenadas' not in servicios_df.columns:
    raise ValueError("El archivo de terrenos debe contener 'Nodos' y el de servicios debe contener 'Coordenadas'.")
if 'Tipo_de_Servicio' not in servicios_df.columns:
    raise ValueError("El archivo de servicios debe contener la columna 'Categoría'.")

# Convertir coordenadas de terrenos a polígonos
terrenos_df['geometry'] = terrenos_df['Nodos'].apply(
    lambda coords: Polygon(eval(coords)) if isinstance(coords, str) else None
)
terrenos_gdf = gpd.GeoDataFrame(terrenos_df, geometry='geometry', crs='EPSG:4326')

# Convertir ubicaciones de servicios a puntos
servicios_df['geometry'] = servicios_df['Coordenadas'].apply(
    lambda loc: Point(eval(loc)) if isinstance(loc, str) else None
)
servicios_gdf = gpd.GeoDataFrame(servicios_df, geometry='geometry', crs='EPSG:4326')

# Verificar intersecciones entre terrenos y servicios
servicios_in_terrenos = gpd.sjoin(servicios_gdf, terrenos_gdf, how='inner', predicate='within')

# Contar servicios por categoría para cada terreno
servicios_count = servicios_in_terrenos.groupby(['ID_Servicio', 'Tipo_de_Servicio']).size().reset_index(name='Cantidad')

# Pivotar para crear una columna por categoría de servicio
servicios_pivot = servicios_count.pivot(index='ID_Servicio', columns='Tipo_de_Servicio', values='Cantidad').fillna(0)

# Unir con los terrenos originales
resultado = terrenos_gdf.merge(servicios_pivot, how='left', left_on='ID_Terreno', right_index=True)
resultado.fillna(0, inplace=True)  # Rellenar valores faltantes con 0

resultado.head(10)


,ID_Terreno,Nodos,Superficie (m²),Precio (€),geometry,Centro Comercial,Colegio,Estación de Metro,Hospital,Parque,Supermercado
0,55229322-9174-4448-aadc-5ab0e8f0414f,"[[40.520874, -3.610398], [40.383694, -3.726544...",1632,204530,"POLYGON ((40.52087 -3.6104, 40.38369 -3.72654,...",0.0,0.0,0.0,0.0,0.0,0.0
1,6745b807-8f10-4a44-8ced-b83871a1ba8c,"[[40.523691, -3.637485], [40.321758, -3.771067...",3507,281418,"POLYGON ((40.52369 -3.63748, 40.32176 -3.77107...",0.0,0.0,0.0,0.0,0.0,0.0
2,0521307b-9c75-49bc-9faa-c1bc263686af,"[[40.513635, -3.846401], [40.584823, -3.745978...",2532,119525,"POLYGON ((40.51364 -3.8464, 40.58482 -3.74598,...",0.0,0.0,0.0,0.0,0.0,0.0
3,f254258c-cc86-45ea-a953-c3f5adee4787,"[[40.354323, -3.5612589999999997], [40.578702,...",4712,369169,"POLYGON ((40.35432 -3.56126, 40.5787 -3.84542,...",0.0,0.0,0.0,0.0,0.0,0.0
4,4be60904-4dfd-4475-9284-00635c67abd7,"[[40.339148, -3.706902], [40.440421, -3.680094...",4554,214653,"POLYGON ((40.33915 -3.7069, 40.44042 -3.68009,...",0.0,0.0,0.0,0.0,0.0,0.0
5,3c149b59-60d3-4948-9ef8-c42d1a7e6667,"[[40.560851, -3.676546], [40.567091, -3.817356...",4567,481964,"POLYGON ((40.56085 -3.67655, 40.56709 -3.81736...",0.0,0.0,0.0,0.0,0.0,0.0
6,8ff8adb4-1f74-4277-ad2f-4e5ad0e2ba51,"[[40.593059, -3.629323], [40.325491, -3.825420...",1711,143771,"POLYGON ((40.59306 -3.62932, 40.32549 -3.82542...",0.0,0.0,0.0,0.0,0.0,0.0
7,90442155-7d78-4bb5-8459-7374c2497727,"[[40.472542, -3.7963500000000003], [40.441128,...",4154,383438,"POLYGON ((40.47254 -3.79635, 40.44113 -3.65824...",0.0,0.0,0.0,0.0,0.0,0.0
8,f11d0867-3028-40e2-83ff-97a7da87a880,"[[40.558513, -3.75168], [40.63989, -3.771283],...",3206,451329,"POLYGON ((40.55851 -3.75168, 40.63989 -3.77128...",0.0,0.0,0.0,0.0,0.0,0.0
9,f52d3524-a419-4609-9923-5ab91d670e70,"[[40.398152, -3.757375], [40.356438, -3.663341...",4577,221048,"POLYGON ((40.39815 -3.75738, 40.35644 -3.66334...",0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# Generar el GeoJSON como string
geojson_data = resultado.to_json()

# Guardar el GeoJSON en un archivo
resultado_path = './data/staging_data/terrenos_con_servicios.json'
with open(resultado_path, 'w') as file:
    file.write(geojson_data)